In [55]:
# Importing necessary libraries
import pymongo
from datetime import datetime, timedelta
from pymongo import InsertOne, DeleteOne, ReplaceOne, UpdateOne
import pandas as pd
import numpy as np  
from pandas.io.json import json_normalize
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso

In [56]:
#importing data from the database
client = pymongo.MongoClient("mongodb+srv://" + "vidit23" + ":" + "dsba123" + "@mvp-bvqf2.mongodb.net/test?retryWrites=true&w=majority")
connectedDB = client['MVP']
collectionName = "Videos"

# Getting the data from the DB
useDate = '13/04/2020'
numDays = 3
query_result = list(connectedDB[collectionName].find({'youtubeId': {'$exists': 1}, 'views.' + useDate: {'$exists': 1}}))

# Normalizing the structure of the dataframe due to nested objects in database
initialSongsDf = pd.io.json.json_normalize(query_result)
print("Shape of incoming data", initialSongsDf.shape)

Shape of incoming data (20880, 91)


In [57]:
initialSongsDf

,_id,name,popularity,artists,albumReleaseDate,albumReleaseDatePrecision,danceability,energy,key,loudness,...,views.07/04/2020.viewCount,views.07/04/2020.likeCount,views.07/04/2020.dislikeCount,views.07/04/2020.favoriteCount,views.07/04/2020.commentCount,views.05/04/2020.viewCount,views.05/04/2020.likeCount,views.05/04/2020.dislikeCount,views.05/04/2020.favoriteCount,views.05/04/2020.commentCount
0,08zJpaUQVi9FrKv2e32Bah,Planez,71,"[{'id': '3KV3p5EY4AvKxOlhGHORLg', 'name': 'Jer...",2015-12-04,day,0.688,0.556,6,-7.738,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,3qocP5BEC1Bu4VVdJPvka6,Something Wonderful,48,"[{'id': '0JXDwBs1sEp6UKoAP58UdF', 'name': 'Sea...",2017-09-15,day,0.602,0.907,0,-4.578,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,5NBbYPv6iAyrHFnSI1VSBa,All The Gold In California,45,"[{'id': '0KzlJXg6A3NVSCkRjecKnS', 'name': 'Lar...",2007-09-04,day,0.650,0.510,7,-10.875,...,58944,621,15,0,19,NaN,NaN,NaN,NaN,NaN
3,7fQuHUNQRLkRevQv6z0Eii,After Midnight,33,"[{'id': '1n2pb9Tsfe4SwAjmUac6YT', 'name': 'Jak...",2016-07-29,day,0.647,0.741,11,-6.655,...,170414,814,35,0,17,NaN,NaN,NaN,NaN,NaN
4,3ulipeC36lUvRzPETyFlm9,Waiting Around To Die,50,"[{'id': '37PywjKNsxHji5F9eFODF2', 'name': 'The...",2003-03-11,day,0.635,0.275,5,-20.085,...,9152411,110843,2028,0,3736,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20875,3L4GsGLgxpHieVQH86H8bv,Bed (feat. Ariana Grande),68,"[{'id': '0hCNtLu0JehylgoiP8L4Gh', 'name': 'Nic...",2018-08-10,day,0.757,0.730,7,-6.304,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20876,436EpHvGoKO3DgaSeD9lW8,No Me Digas Que No,54,"[{'id': '7qG3b048QCHVRO5Pv1T5lw', 'name': 'Enr...",2010-01-01,day,0.734,0.751,9,-2.716,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20877,0YkJU6TjjwzsVXUPRIO0DB,Make Me Better,60,"[{'id': '0YWxKQj2Go9CGHCp77UOyy', 'name': 'Fab...",2007-01-01,day,0.607,0.599,11,-6.886,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20878,3FQJq6X5IHjFyncHoouuRA,Le plus beau de tes rêves (par Anggun),2,"[{'id': '1Z7gkVuSpDsSuRxohBUISi', 'name': 'Mar...",2015-10-16,day,0.505,0.522,1,-6.523,...,389,13,0,0,2,382,13,0,0,2


In [58]:
# dropping views columns we arent considering
useDateEditFormat = datetime.strptime(useDate, "%d/%m/%Y")
viewsColumnsToKeep = []
for addDay in range(numDays+1):
    nextDay = (useDateEditFormat + timedelta(days=addDay)).strftime('%d/%m/%Y')
    viewsColumnsToKeep += ['views.' + nextDay + '.viewCount',
                           'views.' + nextDay + '.likeCount',
                           'views.' + nextDay + '.dislikeCount',
                           'views.' + nextDay + '.commentCount'] 
columnsToKeep = ['popularity', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 
                 'instrumentalness', 'liveness', 'valence', 'tempo', 'duration_ms', 'time_signature']

essentialSongsDf = initialSongsDf[columnsToKeep + viewsColumnsToKeep]

# Dropping all the null columns
essentialSongsDf = essentialSongsDf.dropna()

# Converting all columns to the Dtype float64
essentialSongsDf = essentialSongsDf.astype('float64')
print("Shape after selecting the essential columns and dropping null values ", essentialSongsDf.shape)

Shape after selecting the essential columns and dropping null values  (245, 29)


In [59]:
 query_result[1].keys()

dict_keys(['_id', 'name', 'popularity', 'artists', 'albumReleaseDate', 'albumReleaseDatePrecision', 'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'type', 'duration_ms', 'time_signature', 'youtubeId', 'views'])

In [60]:
essentialSongsDf.columns

Index(['popularity', 'energy', 'key', 'loudness', 'mode', 'speechiness',
       'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo',
       'duration_ms', 'time_signature', 'views.13/04/2020.viewCount',
       'views.13/04/2020.likeCount', 'views.13/04/2020.dislikeCount',
       'views.13/04/2020.commentCount', 'views.14/04/2020.viewCount',
       'views.14/04/2020.likeCount', 'views.14/04/2020.dislikeCount',
       'views.14/04/2020.commentCount', 'views.15/04/2020.viewCount',
       'views.15/04/2020.likeCount', 'views.15/04/2020.dislikeCount',
       'views.15/04/2020.commentCount', 'views.16/04/2020.viewCount',
       'views.16/04/2020.likeCount', 'views.16/04/2020.dislikeCount',
       'views.16/04/2020.commentCount'],
      dtype='object')

In [61]:
essentialSongsDf

,popularity,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,...,views.14/04/2020.dislikeCount,views.14/04/2020.commentCount,views.15/04/2020.viewCount,views.15/04/2020.likeCount,views.15/04/2020.dislikeCount,views.15/04/2020.commentCount,views.16/04/2020.viewCount,views.16/04/2020.likeCount,views.16/04/2020.dislikeCount,views.16/04/2020.commentCount
0,71.0,0.556,6.0,-7.738,1.0,0.2640,0.70000,0.000002,0.1080,0.416,...,32045.0,10240.0,113803263.0,653903.0,32058.0,10240.0,113849498.0,654115.0,32065.0,10240.0
1,48.0,0.907,0.0,-4.578,0.0,0.0495,0.00015,0.000007,0.0812,0.642,...,39.0,170.0,204188.0,2584.0,39.0,170.0,204269.0,2584.0,39.0,170.0
2,45.0,0.510,7.0,-10.875,1.0,0.0339,0.45200,0.000000,0.0844,0.687,...,16.0,22.0,59425.0,628.0,16.0,22.0,59481.0,629.0,16.0,22.0
3,33.0,0.741,11.0,-6.655,1.0,0.0250,0.01410,0.000010,0.2920,0.547,...,35.0,17.0,170766.0,814.0,35.0,17.0,170835.0,814.0,35.0,17.0
4,50.0,0.275,5.0,-20.085,0.0,0.0394,0.49600,0.143000,0.1090,0.323,...,2034.0,3766.0,9181929.0,111407.0,2034.0,3769.0,9185510.0,111479.0,2033.0,3772.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
315,47.0,0.904,5.0,-4.377,0.0,0.0746,0.00055,0.000000,0.3790,0.840,...,2118.0,3562.0,11813640.0,88413.0,2118.0,3562.0,11816448.0,88442.0,2118.0,3562.0
316,1.0,0.415,7.0,-10.019,0.0,0.1300,0.28300,0.000000,0.0883,0.339,...,1826.0,1688.0,6806490.0,47813.0,1826.0,1688.0,6808041.0,47826.0,1826.0,1690.0
317,29.0,0.758,2.0,-5.602,1.0,0.0343,0.20800,0.000310,0.0957,0.524,...,17.0,11.0,39301.0,271.0,17.0,11.0,39425.0,271.0,17.0,11.0
319,0.0,0.101,2.0,-9.165,1.0,0.0420,0.95700,0.000004,0.1910,0.393,...,3.0,1.0,3699.0,36.0,3.0,1.0,3714.0,36.0,3.0,1.0


In [71]:
# [[5,6,7,8,{'13', '14', '15', '16', '17'}]]

# [[5,6,7,8,{'13', '14', '15'}],
# [5,6,7,8,{'14', '15', '16'}],
# [5,6,7,8,{'15', '16', '17'}]]
column_names = ['_id','popularity', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 
                 'instrumentalness', 'liveness', 'valence', 'tempo', 'duration_ms', 'time_signature']
songsByDay = pd.DataFrame(columns = column_names)

In [72]:
songsByDay = essentialSongsDf[columnsToKeep]
songsByDay

,popularity,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,71.0,0.556,6.0,-7.738,1.0,0.2640,0.70000,0.000002,0.1080,0.416,129.336,240320.0,4.0
1,48.0,0.907,0.0,-4.578,0.0,0.0495,0.00015,0.000007,0.0812,0.642,116.068,178532.0,4.0
2,45.0,0.510,7.0,-10.875,1.0,0.0339,0.45200,0.000000,0.0844,0.687,106.003,172867.0,5.0
3,33.0,0.741,11.0,-6.655,1.0,0.0250,0.01410,0.000010,0.2920,0.547,105.006,236253.0,4.0
4,50.0,0.275,5.0,-20.085,0.0,0.0394,0.49600,0.143000,0.1090,0.323,96.737,313880.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
315,47.0,0.904,5.0,-4.377,0.0,0.0746,0.00055,0.000000,0.3790,0.840,92.948,185080.0,4.0
316,1.0,0.415,7.0,-10.019,0.0,0.1300,0.28300,0.000000,0.0883,0.339,126.475,248693.0,4.0
317,29.0,0.758,2.0,-5.602,1.0,0.0343,0.20800,0.000310,0.0957,0.524,131.308,362105.0,4.0
319,0.0,0.101,2.0,-9.165,1.0,0.0420,0.95700,0.000004,0.1910,0.393,134.915,29000.0,4.0


In [73]:
rows =[]
dayDetailsDict=[]
columnsToKeep = ['popularity', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 
                 'instrumentalness', 'liveness', 'valence', 'tempo', 'duration_ms', 'time_signature']
for addDay in range(numDays+1):
    nextDay = (useDateEditFormat + timedelta(days=addDay)).strftime('%d/%m/%Y')
    viewsColumnsToKeep += ['views.' + nextDay + '.viewCount',
                           'views.' + nextDay + '.likeCount',
                           'views.' + nextDay + '.dislikeCount',
                           'views.' + nextDay + '.commentCount'] 
dayDetailsDF = essentialSongsDf[viewsColumnsToKeep]
dayDetailsDict.append(dayDetailsDF.to_dict('index'))
dayDetailsDict

/Users/nandhitharaghuram/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  if sys.path[0] == '':


[{0: {'views.13/04/2020.viewCount': 113716493.0,
   'views.13/04/2020.likeCount': 653429.0,
   'views.13/04/2020.dislikeCount': 32038.0,
   'views.13/04/2020.commentCount': 10237.0,
   'views.14/04/2020.viewCount': 113760431.0,
   'views.14/04/2020.likeCount': 653676.0,
   'views.14/04/2020.dislikeCount': 32045.0,
   'views.14/04/2020.commentCount': 10240.0,
   'views.15/04/2020.viewCount': 113803263.0,
   'views.15/04/2020.likeCount': 653903.0,
   'views.15/04/2020.dislikeCount': 32058.0,
   'views.15/04/2020.commentCount': 10240.0,
   'views.16/04/2020.viewCount': 113849498.0,
   'views.16/04/2020.likeCount': 654115.0,
   'views.16/04/2020.dislikeCount': 32065.0,
   'views.16/04/2020.commentCount': 10240.0},
  1: {'views.13/04/2020.viewCount': 203985.0,
   'views.13/04/2020.likeCount': 2582.0,
   'views.13/04/2020.dislikeCount': 39.0,
   'views.13/04/2020.commentCount': 170.0,
   'views.14/04/2020.viewCount': 204101.0,
   'views.14/04/2020.likeCount': 2583.0,
   'views.14/04/2020.dis

In [90]:
daysColumn = list(dayDetailsDict[0].values())
len(daysColumn)
type(daysColumn)

list

In [91]:
songsByDay['days'] = daysColumn

/Users/nandhitharaghuram/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [94]:
songsByDay['days'][0]

{'views.13/04/2020.viewCount': 113716493.0,
 'views.13/04/2020.likeCount': 653429.0,
 'views.13/04/2020.dislikeCount': 32038.0,
 'views.13/04/2020.commentCount': 10237.0,
 'views.14/04/2020.viewCount': 113760431.0,
 'views.14/04/2020.likeCount': 653676.0,
 'views.14/04/2020.dislikeCount': 32045.0,
 'views.14/04/2020.commentCount': 10240.0,
 'views.15/04/2020.viewCount': 113803263.0,
 'views.15/04/2020.likeCount': 653903.0,
 'views.15/04/2020.dislikeCount': 32058.0,
 'views.15/04/2020.commentCount': 10240.0,
 'views.16/04/2020.viewCount': 113849498.0,
 'views.16/04/2020.likeCount': 654115.0,
 'views.16/04/2020.dislikeCount': 32065.0,
 'views.16/04/2020.commentCount': 10240.0}

In [95]:
songsByDay

,popularity,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,days
0,71.0,0.556,6.0,-7.738,1.0,0.2640,0.70000,0.000002,0.1080,0.416,129.336,240320.0,4.0,"{'views.13/04/2020.viewCount': 113716493.0, 'v..."
1,48.0,0.907,0.0,-4.578,0.0,0.0495,0.00015,0.000007,0.0812,0.642,116.068,178532.0,4.0,"{'views.13/04/2020.viewCount': 203985.0, 'view..."
2,45.0,0.510,7.0,-10.875,1.0,0.0339,0.45200,0.000000,0.0844,0.687,106.003,172867.0,5.0,"{'views.13/04/2020.viewCount': 59338.0, 'views..."
3,33.0,0.741,11.0,-6.655,1.0,0.0250,0.01410,0.000010,0.2920,0.547,105.006,236253.0,4.0,"{'views.13/04/2020.viewCount': 170669.0, 'view..."
4,50.0,0.275,5.0,-20.085,0.0,0.0394,0.49600,0.143000,0.1090,0.323,96.737,313880.0,4.0,"{'views.13/04/2020.viewCount': 9175013.0, 'vie..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
315,47.0,0.904,5.0,-4.377,0.0,0.0746,0.00055,0.000000,0.3790,0.840,92.948,185080.0,4.0,"{'views.13/04/2020.viewCount': 11808619.0, 'vi..."
316,1.0,0.415,7.0,-10.019,0.0,0.1300,0.28300,0.000000,0.0883,0.339,126.475,248693.0,4.0,"{'views.13/04/2020.viewCount': 6803571.0, 'vie..."
317,29.0,0.758,2.0,-5.602,1.0,0.0343,0.20800,0.000310,0.0957,0.524,131.308,362105.0,4.0,"{'views.13/04/2020.viewCount': 39066.0, 'views..."
319,0.0,0.101,2.0,-9.165,1.0,0.0420,0.95700,0.000004,0.1910,0.393,134.915,29000.0,4.0,"{'views.13/04/2020.viewCount': 3652.0, 'views...."
